In [1]:
import json
import dinopy

In [11]:
with open("revcomp_interval/t7.fasta-concatenate.json") as f_, open("revcomp_interval/t7.fasta-mCGR.json") as g_:
    conc = json.load(f_)
    mcgr = json.load(g_)

## Full implementation

In [12]:
failed_counter = 0
sucess_counter = 0

freader = dinopy.FastaReader("eval_50_gc/faust.fasta")
for sequence in freader.reads():
    flag = 0
    dec = sequence.decode()
    half = int(len(dec)/2)
    pre, post = dec[:half], dec[half:]
    if not pre in mcgr['mCGR(row,col)'].keys() or not post in mcgr['mCGR(row,col)'].keys():
        failed_counter += 1
        #print("Is not in allowed sequences")
        flag = 1
        #pass
    else:
        row, col = mcgr['mCGR(row,col)'][pre]
        hits_pre = set()
        for key in conc["prepending(row:col)"].keys():
            if str(row) in conc["prepending(row:col)"][key].keys():
                if col in conc["prepending(row:col)"][key][str(row)]:
                    hits_pre.update([key])

        m_hits = set()
        for hit in hits_pre:
            if str(hit) in conc["motif"].keys():
                m_hits.update(conc["motif"][hit])

        row, col = mcgr['mCGR(row,col)'][post]
        hits_post = set()
        for mhit in m_hits:
            if str(row) in conc["appending(row:col)"][mhit].keys() and col in conc["appending(row:col)"][mhit][str(row)]:
                #print("pre and postfix lead to a forbidden motif")
                failed_counter += 1
                flag = 1
                break
    if not flag:
        sucess_counter += 1

In [13]:
sucess_counter

478315

In [14]:
failed_counter

108325

In [27]:
failed_counter/(failed_counter + sucess_counter)

0.16971055502522842

## Testing

In [4]:
dec = "CGCTATGAATCG"

failed_counter = 0
sucess_counter = 0

half = int(len(dec)/2)
pre, post = dec[:half], dec[half:]
if not pre in mcgr['mCGR(row,col)'].keys() or not post in mcgr['mCGR(row,col)'].keys():
    failed_counter += 1
    print("Is not in allowed sequences")
    #pass
else:
    row, col = mcgr['mCGR(row,col)'][pre]
    hits_pre = set()
    for key in conc["prepending(row:col)"].keys():
        if str(row) in conc["prepending(row:col)"][key].keys():
            if col in conc["prepending(row:col)"][key][str(row)]:
                print("jep")
                print(key)
                print("-----")
                hits_pre.update([key])
                
    m_hits = set()
    for hit in hits_pre:
        if str(hit) in conc["motif"].keys():
            m_hits.update(conc["motif"][hit])
    
    row, col = mcgr['mCGR(row,col)'][post]
    hits_post = set()
    for mhit in m_hits:
        if str(row) in conc["appending(row:col)"][mhit].keys() and col in conc["appending(row:col)"][mhit][str(row)]:
            print("pre and postfix lead to a forbidden motif")
            failed_counter += 1
            break
    sucess_counter += 1

jep
AT
-----
pre and postfix lead to a forbidden motif
